# Algoritmos de Policy Gradient

## REINFORCE

In [1]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

from xlsx_generator import XlsxGenerator
# from ..xlsx_read import XlsxReader

In [2]:
# --- Hiperparâmetros ---
LEARNING_RATE = 0.01
GAMMA = 0.99         # Fator de desconto para recompensas futuras
N_EPISODES = 1000


# --- Passo 1: Definir a Rede de Política (O "Ator") ---
# Esta rede neural decide qual ação tomar dado um estado.
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        
        # 4 entradas (estado) -> 128 neurônios -> 2 saídas (logits da ação)
        self.fc1 = nn.Linear(state_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, action_dim) # Saída são os logits
    
    def forward(self, state):
        x = self.fc1(state)
        x = self.relu(x)
        
        logits = self.fc2(x)
        return logits
    
# --- Passo 2: Função para selecionar ação e calcular log_prob ---
def select_action(network, state):
    state = torch.tensor(state, dtype=torch.float32)
    logits = network(state)
    
    # Cria uma distribuição categórica (ex: [0.6, 0.4]) a partir dos logits
    # O PyTorch aplica o softmax internamente
    dist = Categorical(logits=logits)
    
    # Amostra uma ação da distribuição (Exploração!)
    action = dist.sample()
    
    # Calcula o log(prob) da ação que foi amostrada.
    # Isso é o log(pi(a|s)) que precisamos para a perda!
    log_prob = dist.log_prob(action)
    
    return action.item(), log_prob
    
# --- Passo 3: Função para calcular os Retornos Descontados (G_t) ---
def calculate_discounted_returns(rewards):
    """
    Calcula o "reward-to-go" (G_t) para cada passo no episódio.
    """
    returns = []
    G_t = 0 # Retorno acumulado
    
    # Itera de trás para frente sobre as recompensas
    for r in reversed(rewards):
        # G_t = r_t + gamma * G_t+1
        G_t = r + GAMMA * G_t
        returns.insert(0, G_t) # Insere no início da lista
        
    # Converte para tensor
    returns = torch.tensor(returns, dtype=torch.float32)
    
    # [TRUQUE DE ESTABILIZAÇÃO]: Normalizar os retornos
    # Isso centraliza os retornos em torno de 0.
    # Vantagens "boas" se tornam > 0, "ruins" < 0.
    returns = (returns - returns.mean()) / (returns.std() + 1e-9) # Evita divisão por zero
    
    return returns

# --- Passo 4: O Loop de Treinamento ---

# Inicializa o ambiente e a rede
env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

policy_net = PolicyNetwork(state_dim, action_dim)
optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)

print("Iniciando treinamento com REINFORCE...")

episode_details = []

for episode in range(N_EPISODES):
    # Listas para armazenar dados do episódio
    rewards = []
    log_probs = []
    
    state, _ = env.reset()
    terminated = False
    truncated = False
    total_reward = 0

    # --- 1. Coletar um episódio completo (Rollout) ---
    while not (terminated or truncated):
        # Seleciona a ação e o log_prob
        action, log_prob = select_action(policy_net, state)
        
        # Executa a ação no ambiente
        next_state, reward, terminated, truncated, _ = env.step(action)
        episode_details.append({
            "episode": episode + 1,
            "step_reward": reward,            
            "total_reward": total_reward + reward,
            "action": action,
            'terminated': terminated,
            'truncated': truncated,
            'next_state': next_state,
            'reward': reward,
            'data': _,
            'type': 'step'
        })
        # Armazena os dados
        rewards.append(reward)
        log_probs.append(log_prob)
        
        state = next_state
        total_reward += reward
    
    # --- 2. Calcular Retornos ---
    returns = calculate_discounted_returns(rewards)

    # --- 3. Calcular a Perda (Loss) ---
    policy_loss = []
    # Itera sobre cada passo (log_prob, G_t)
    for log_prob, G_t in zip(log_probs, returns):
        # A perda é -log(pi(a|s)) * G_t
        # O sinal negativo é porque o otimizador TENTA MINIMIZAR.
        # Minimizar -[log_prob * G_t] é o mesmo que MAXIMIZAR [log_prob * G_t].
        policy_loss.append(-log_prob * G_t)
        
    # Soma as perdas de todos os passos do episódio
    # Precisamos empilhar os tensores antes de somar
    loss = torch.stack(policy_loss).sum()
    episode_details.append({
        "episode": episode + 1,
        "step_reward": None,            
        "total_reward": total_reward,
        "action": None,
        'terminated': terminated,
        'truncated': truncated,
        'next_state': None,
        'reward': None,
        'data': None,
        'type': 'loss',
        'loss': loss.item()
    })
    # --- 4. Atualizar a Rede ---
    optimizer.zero_grad() # Zera os gradientes
    loss.backward()       # Calcula os gradientes (Backpropagation)
    optimizer.step()      # Aplica os gradientes
    if (episode + 1) % 100 == 0:
        print(f"Episódio {episode+1}/{N_EPISODES}, Recompensa Total: {total_reward}")

env.close()
print("Treinamento REINFORCE concluído.")


Iniciando treinamento com REINFORCE...
Episódio 100/1000, Recompensa Total: 207.0
Episódio 200/1000, Recompensa Total: 500.0
Episódio 300/1000, Recompensa Total: 18.0
Episódio 400/1000, Recompensa Total: 500.0
Episódio 500/1000, Recompensa Total: 500.0
Episódio 600/1000, Recompensa Total: 164.0
Episódio 700/1000, Recompensa Total: 500.0
Episódio 800/1000, Recompensa Total: 224.0
Episódio 900/1000, Recompensa Total: 402.0
Episódio 1000/1000, Recompensa Total: 500.0
Treinamento REINFORCE concluído.


In [4]:
xlsx = XlsxGenerator("outputs/reinforce_training_log.xlsx")
xlsx.add_dataframes("training_log", episode_details)
xlsx.save()

'/home/yang/projects/inf2070-gym/inf2070-studies/notebooks/outputs/reinforce_training_log_20251026_165136.xlsx'